In [1]:
import os
from BH.feature_functions import *

GPU_NUM = "0"
num_epochs = 50
batch_size = 128

use_ppath = False
column_info = "original"

graph_deg = 5
num_layers = graph_deg
num_features = 256
feature_list = {
    'constant':    (False, constant_feature),
    'column':      (False, column_indicator),
    'norm_column': (True, normalized_column_indicator), #   Feauture
    'norm_column_rev': (False, normalized_column_rev_indicator),
}
connected = True
UPTO = False

direction = "222"

shape_indicator = {
    'all_with_all_row_connectedness_criterion': (False, ),
    'all_with_inductive_connectedness_criterion': (True, ),
    '2row_less': (False, ),
    '2row_less_with_all_row_connectedness_criterion': (False, ),
    '2row_less_with_inductive_connectedness_criterion': (False, ),
    '3row_less_with_all_row_connectedness_criterion': (False, ),
    '3row_less_with_inductive_connectedness_criterion': (False, ),
}

shape= {
    'all': (False, ),# 3개다 
    '2row_less': (True, ),
    '3row_less': (False, ),
}
filter_indicator = {
    'with_all_row_connectedness_criterion':(False,), 
    'with_inductive_connectedness_criterion':(False,),
}


use_pretrained_weights = False
save_trained_weights = True

step_size = 0.00001
train_fraction = .8

DIR_PATH = f'/Data/Ptab/n={graph_deg}'
MODEL_DIR = './trained_models'
MODEL_FILE = os.path.join(MODEL_DIR, f'parameters_{graph_deg}_{num_layers}_{num_features}')

for key in shape_indicator:
    if shape_indicator[key][0] == True:
        DIR_PATH += f'_{key}'
        MODEL_FILE += f'_{key}'


if connected == True:
    DIR_PATH += "_connected"
    MODEL_FILE += "_connected"
elif connected == False:
    DIR_PATH += "_disconnected"
    MODEL_FILE += "_disconnected"
if UPTO == True:
    DIR_PATH += "_UPTO"
    MODEL_FILE += "_UPTO"
if not column_info == "original":
    DIR_PATH += column_info
    MODEL_FILE += column_info
if use_ppath:
    DIR_PATH += "_ppath"    
    MODEL_FILE += "_ppath"
    
for key in feature_list.keys():
    if feature_list[key][0] == True:
        MODEL_FILE += f'_{key}'
MODEL_FILE += f'_{direction}'
MODEL_FILE += '.pickle'



<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import functools
import enum
import os
from BH.data_loader import *
# from BH.generate_data import *
#from BH.data_loader import *
#from BH.generate_data import *
# from training_info import *

import pickle

from CustomDataset import *
from GCN_model import *

from torch_geometric.loader import DataLoader


os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NUM
device = "cuda:" + GPU_NUM


print("Loading input data...")
full_dataset, train_dataset, test_dataset = load_input_data(DIR_PATH)

node_dim = num_features
edge_dim = 8
graph_deg = graph_deg
depth = num_layers

test_dataset = CustomDataset(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

train_dataset = CustomDataset(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = pastGCN().to(device)
if use_pretrained_weights == True:
    try:
        with open(MODEL_FILE, 'rb') as f:
          model, max_accuracy, min_loss = pickle.load(f)
          model.to(device)
    except:
        print("There is no trained model")
        use_pretrained_weights = False
if use_pretrained_weights == False:
    model = GCN_multi_conv(graph_deg, depth, node_dim, direction).to(device)
    max_accuracy = 0
    min_loss = 100
# data = batch.to(device)
# torch.nn.init.xavier_normal(model)
loss_function = torch.nn.CrossEntropyLoss()
# loss_function = torch.nn.NLLLoss()
# loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=step_size, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.993 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

/root/anaconda3/envs/mpnn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading input data...
Generating data from the directory /Data/Ptab/n=5_all_with_inductive_connectedness_criterion_connected


In [3]:
for epoch in range(1):
    # Training phase
    model.train()
    for batch in train_loader:
        batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        log_probs = torch.log(out)
        
        loss = loss_function(log_probs, batch.y)
#         break
        loss.backward()
        optimizer.step()
        print(loss)
    scheduler.step()
    print(loss)
    
    # Evaluation phase
    model.eval()
    correct = 0
    total = 0
#     with torch.no_grad():
#         for batch in test_loader:
#             batch.to(device)
#             outputs = model(batch)
# #             _,predicted = torch.max(outputs.data, 1)
#             predicted = outputs
#             total += batch.y.size(0)
# #             correct += (predicted == batch.y).sum().item()
#             correct += ((predicted - batch.y)**2<0.1).sum().item()
    with torch.no_grad():  # Disable gradient calculation
        total = 0
        correct = 0
        for batch in test_loader:
            # Move batch to the appropriate device (GPU or CPU)
            batch.to(device)

            # Forward pass: get model predictions
            outputs = model(batch)

            # Predicted classes are the indices of the max logits along the class dimension
            _, predicted = torch.max(outputs.data, 1)  # Shape: (batch_size)

            # Increment the total number of samples
            total += batch.y.size(0)

            # Compare predicted labels with ground truth labels
            correct += (predicted == batch.y).sum().item()  # Count the number of correct predictions

    # Compute the accuracy as a percentage
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    # Compute accuracy
    accuracy = correct / total
    loss = float(loss.item())

    print("Epoch [{}/{}], Accuracy: {:.2%}, Loss: {:.15f}".format(epoch + 1, num_epochs, accuracy, loss))

    if (accuracy > max_accuracy or (accuracy == max_accuracy and loss < min_loss)) and save_trained_weights:
        max_accuracy = accuracy
        min_loss = loss
        with open(MODEL_FILE, 'wb') as f:
            pickle.dump((model, max_accuracy, min_loss), f)

/root/Min/Ptab/CustomDataset.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/torch/csrc/utils/tensor_new.cpp:201.)
  edge_index = torch.tensor([self.rows[idx], self.cols[idx]], dtype=torch.long)


tensor(0.9219, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 41.94%
Epoch [1/50], Accuracy: 41.94%, Loss: nan


In [4]:
ㅁ

NameError: name 'ᄆ' is not defined

In [ ]:
for batch in train_loader:
    print(batch)
    batch.to(device)
    out = model(batch)
    out_single = model.GCN_single(batch)
    break

In [ ]:
torch.log(out)

In [ ]:
loss = loss_function(torch.log(out), batch.y)

In [ ]:
loss

In [ ]:
batch = batch.batch

In [ ]:
batch=batch[::model.graph_deg]

In [ ]:
x =out_single

In [ ]:
x.shape

In [ ]:
batch.shape

In [ ]:
# Flatten A and ri to 1D tensors for easier manipulation
A = x.view(-1)
ri = batch.view(-1)
batch_size = batch[-1]+1

# Count occurrences of each row index in ri to determine column sizes
row_counts = torch.bincount(ri, minlength=batch_size)

# Determine the maximum number of columns needed
max_columns = row_counts.max().item()

# Initialize tensor B with -inf to indicate empty values
B = torch.full((batch_size, max_columns), -float('inf'),device = device)

# Create an empty tensor to keep track of the next available column index for each row
col_indices = torch.zeros_like(ri,device=device)

# Use row_counts to generate column indices
for i in range(batch_size):
    col_indices[ri == i] = torch.arange(row_counts[i],device = device)

# Place the values from A into B using advanced indexing
B[ri, col_indices] = A


In [ ]:
B = torch.sigmoid(B)
B_complement = 1 - B

# Stack B and B_complement along a new dimension
# This creates a new tensor with shape (64, max_columns, 2)
stacked_tensor = torch.stack((B_complement,B), dim=-1)

print(stacked_tensor.shape)

In [ ]:
# Loop through the remaining slices
result = stacked_tensor[:,0,:]
for i in range(1, stacked_tensor.size(1)):
    # Extract the next slice to serve as the kernel, which has shape (1, 1, 64, 2)
    kernel_2d = stacked_tensor[:,i,:].unsqueeze(1)
    kernel_2d_flipped = torch.flip(kernel_2d, dims=[-1])

    # Reshape the input for Conv1d: treat each row as a separate channel
    input_for_conv1d = result.unsqueeze(0)  # Shape becomes (1, H, W)

    # Apply Conv1d with groups=H to perform independent convolution on each row
    # in_channels = H (number of rows), out_channels = H (each row produces its own output),
    # groups=H ensures each row is convolved with its own corresponding kernel row
    result = F.conv1d(input_for_conv1d, kernel_2d_flipped, groups=batch_size, padding=1)

    # Remove the extra dimension to get the final result
    result = result.squeeze()

# The final result will be in `result`
print(result.shape)

In [ ]:
result.shape

In [ ]:
out.shape

In [ ]:
batch.to(device)
optimizer.zero_grad()
out = model(batch)

In [ ]:
out

In [ ]:
batch.y